In [1]:
import pandas as pd

# --- 1. CSV を読み込む（パスは自分の環境に合わせて変えてください） ---

df = pd.read_csv(r'C:\Users\akafu\analytics\aim_machi\aggregated.csv') # 例: './data/aggregated.csv'

# --- 2. 「1/xxx」表記を数値化 ---
df['RB_denominator'] = df['RB率'].str.extract(r'1/(\d+)').astype(float)
df['RB_frequency']  = 1 / df['RB_denominator']

# --- 3. RB_frequency で降順ソート （REG率が高い順）
df_sorted = df.sort_values('RB_frequency', ascending=False)

# --- 表示　---
df_sorted.head()  



,台番,G数,BB,RB,差枚,出率,合成,BB率,RB率,RB_denominator,RB_frequency
11,555,552144,2056,1870,39191,102.4%,1/141,1/269,1/295,295.0,0.003390
20,564,444413,1704,1401,25225,101.9%,1/143,1/261,1/317,317.0,0.003155
19,563,459745,1650,1452,-9012,99.3%,1/148,1/279,1/317,317.0,0.003155
4,548,459138,1715,1440,11068,100.8%,1/146,1/268,1/319,319.0,0.003135
28,593,431704,1618,1352,14195,101.0%,1/145,1/267,1/319,319.0,0.003135


In [ ]:
import os
import pandas as pd
from datetime import datetime, timedelta

# ===== 設定 =====
DATA_DIR   = r"C:\Users\akafu\analytics\aim_machi\data2"                  # 読み込み元（日次CSV）
OUTPUT_DIR = r"C:\Users\akafu\analytics\aim_machi\notebook\result"       # 出力先（まとめCSV）
START_DATE = "2025-6-13"   # ←開始日
END_DATE   = "2025-9-15"   # ←終了日

# 抽出条件
G_MIN = 4000
RATE_MAX_DEN = 260# REG/RB率の「1/xxx」の xxx（分母）がこれ以下ならOK
SAMAI_UPPER = 1000

SKIP_MISSING_DAYS = True  # 当日のCSVが無い日はスキップ（Falseでエラー）

# ===== ユーティリティ =====
def weekday_str(d: datetime) -> str:
    return d.strftime("%a")  # Mon/Tue/...

def compose_path(d: datetime) -> str:
    # aim-YYYY-MM-DD-曜_with_grape.csv を読む
    return os.path.join(
        DATA_DIR,
        f"aim-{d.strftime('%Y-%m-%d')}-{weekday_str(d)}_with_grape.csv"
    )

def _clean_int_col(df: pd.DataFrame, col: str) -> None:
    """カンマ・全角マイナス・空白などを除去して、nullable Int64 に変換"""
    if col not in df.columns:
        return
    s = (df[col].astype(str)
                 .str.replace("\u2212", "-", regex=False)   # 全角マイナス → 半角
                 .str.replace(",", "", regex=False)          # カンマ除去
                 .str.replace(r"\s+", "", regex=True))       # 空白除去
    s = s.str.extract(r"([+-]?\d+)", expand=False)           # 先頭の符号付き整数だけ抽出
    df[col] = pd.to_numeric(s, errors="coerce").astype("Int64")

def read_day(d: datetime) -> pd.DataFrame:
    """1日のCSVを読み、率の分母抽出＋主要数値列を Int64 正規化して返す"""
    path = compose_path(d)
    if not os.path.exists(path):
        raise FileNotFoundError(path)

    df = pd.read_csv(path)

    # 率の分母（REG率優先、無ければRB率）を数値化
    rate_col = "REG率" if "REG率" in df.columns else ("RB率" if "RB率" in df.columns else None)
    if rate_col is None:
        raise KeyError("REG率 も RB率 も見つかりませんでした。")
    df["rate_denominator"] = pd.to_numeric(
        df[rate_col].astype(str).str.extract(r"1/(\d+)", expand=False),
        errors="coerce"
    )  # floatのままでOK

    # 主要数値列を安全に Int64 化
    for c in ["G数", "差枚", "BB", "RB"]:
        _clean_int_col(df, c)

    return df

def filter_by_conditions(df: pd.DataFrame) -> pd.DataFrame:
    """抽出条件（G数、rate_denominator、差枚）でフィルタ"""
    for col in ["G数", "差枚", "rate_denominator"]:
        if col not in df.columns:
            raise KeyError(f"列 {col} が見つかりません。")

    cond_g    = (df["G数"] >= G_MIN).fillna(False)
    cond_rate = (df["rate_denominator"] <= RATE_MAX_DEN).fillna(False)
    cond_s    = (df["差枚"] < SAMAI_UPPER).fillna(False)

    out = df.loc[cond_g & cond_rate & cond_s].copy()
    # 見やすく：良い順（分母が小さいほど良い）→ G数大きい順
    out = out.sort_values(["rate_denominator", "G数"], ascending=[True, False])
    return out

def attach_next_day_rows(today_date: datetime) -> pd.DataFrame:
    """当日の抽出結果に、翌日の同台番のカラムを _next で横付け"""
    df_today = read_day(today_date)
    filtered = filter_by_conditions(df_today)

    next_date = today_date + timedelta(days=1)

    # 翌日のCSV（無ければ台番だけの枠を作り、結合後はNaNのまま残す）
    try:
        df_next = read_day(next_date)
    except FileNotFoundError:
        df_next = pd.DataFrame({"台番": filtered["台番"].unique()})

    # 翌日から拾う列
    keep_cols = ["台番", "差枚", "G数", "BB", "RB", "出率", "BB率", "RB率", "合成"]
    exist_cols = [c for c in keep_cols if c in df_next.columns]
    df_next_small = df_next[exist_cols].copy()
    df_next_small = df_next_small.rename(columns={c: f"{c}_next" for c in exist_cols if c != "台番"})

    merged = filtered.merge(df_next_small, on="台番", how="left")
    merged.insert(0, "date", today_date.strftime("%Y-%m-%d"))
    merged.insert(1, "next_date", next_date.strftime("%Y-%m-%d"))
    return merged

def daterange(start_date: datetime, end_date: datetime):
    d = start_date
    while d <= end_date:
        yield d
        d += timedelta(days=1)

# ===== メイン（まとめのみ保存） =====
def run_range(start_date_str: str, end_date_str: str) -> pd.DataFrame:
    start = datetime.strptime(start_date_str, "%Y-%m-%d")
    end   = datetime.strptime(end_date_str,   "%Y-%m-%d")

    all_rows = []
    for d in daterange(start, end):
        path = compose_path(d)
        if not os.path.exists(path):
            msg = f"[SKIP] {os.path.basename(path)} が見つかりません。"
            if SKIP_MISSING_DAYS:
                print(msg)
                continue
            else:
                raise FileNotFoundError(msg)
        try:
            day_result = attach_next_day_rows(d)
            all_rows.append(day_result)
        except Exception as e:
            print(f"[ERROR] {d.date()} の処理でエラー: {e}")

    if not all_rows:
        print("[INFO] 条件に一致する行がありませんでした。")
        return pd.DataFrame()

    total = pd.concat(all_rows, ignore_index=True)

    # *_next を整数（nullable Int64）に揃える（欠損は <NA> のまま）
    for c in ["差枚_next", "G数_next", "BB_next", "RB_next"]:
        if c in total.columns:
            total[c] = pd.to_numeric(total[c], errors="coerce").astype("Int64")

    # 出力先フォルダ（notebook\result）に保存
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    out_total = os.path.join(
        OUTPUT_DIR,
        f"filter-next-aim-{start:%Y%m%d} {end:%Y%m%d}.csv"
    )
    total.to_csv(out_total, index=False, encoding="utf-8-sig")
    print(f"[SAVE] {out_total} (rows={len(total)})")
    return total

if __name__ == "__main__":
    _ = run_range(START_DATE, END_DATE)


[ERROR] 2025-06-30 の処理でエラー: You are trying to merge on int64 and object columns for key '台番'. If you wish to proceed you should use pd.concat
[SAVE] C:\Users\akafu\analytics\aim_machi\notebook\result\filter-next-aim-20250613 20250915.csv (rows=267)


In [ ]:
import os
import pandas as pd
from datetime import datetime, timedelta

# ===== 設定 =====
DATA_DIR   = r"C:\Users\akafu\analytics\aim_machi\data"        # 読み込み元（日次CSV）
OUTPUT_DIR = r"C:\Users\akafu\analytics\aim_machi\notebook\result"      # 出力先（まとめCSV）
START_DATE = "2025-06-13"# ←開始日
END_DATE   = "2025-09-15" # ←終了日

# 抽出条件
G_MIN = 0
RATE_MAX_DEN = 100000    # REG/RB率の「1/xxx」の xxx（分母）がこれ以下ならOK
SAMAI_UPPER = 10000     

# 特定の台
SPECIFIC_DAI_BAN_LIST = ["593"] 

SKIP_MISSING_DAYS = True  # 当日のCSVが無い日はスキップ（Falseでエラー）

# ===== ユーティリティ =====
def weekday_str(d: datetime) -> str:
    return d.strftime("%a")  # Mon/Tue/...

def compose_path(d: datetime) -> str:
    return os.path.join(DATA_DIR, f"aim-{d.strftime('%Y-%m-%d')}-{weekday_str(d)}.csv")

def _clean_int_col(df: pd.DataFrame, col: str) -> None:
    """カンマ・全角マイナス・空白などを除去して、nullable Int64 に変換"""
    if col not in df.columns:
        return
    s = (df[col].astype(str)
                 .str.replace("\u2212", "-", regex=False)   # 全角マイナス → 半角
                 .str.replace(",", "", regex=False)          # カンマ除去
                 .str.replace(r"\s+", "", regex=True))       # 空白除去
    s = s.str.extract(r"([+-]?\d+)", expand=False)          # 先頭の符号付き整数だけ抽出
    df[col] = pd.to_numeric(s, errors="coerce").astype("Int64")

def read_day(d: datetime) -> pd.DataFrame:
    """1日のCSVを読み、率の分母抽出＋主要数値列を Int64 正規化して返す"""
    path = compose_path(d)
    if not os.path.exists(path):
        raise FileNotFoundError(path)

    df = pd.read_csv(path)

    # 率の分母（REG率優先、無ければRB率）を数値化
    rate_col = "REG率" if "REG率" in df.columns else ("RB率" if "RB率" in df.columns else None)
    if rate_col is None:
        raise KeyError("REG率 も RB率 も見つかりませんでした。")
    df["rate_denominator"] = pd.to_numeric(
        df[rate_col].astype(str).str.extract(r"1/(\d+)", expand=False),
        errors="coerce"
    )  # floatのままでOK

    # 主要数値列を安全に Int64 化
    for c in ["G数", "差枚", "BB", "RB"]:
        _clean_int_col(df, c)

    return df

def filter_by_conditions(df: pd.DataFrame) -> pd.DataFrame:
    """抽出条件（G数、rate_denominator、差枚、台番）でフィルタ"""
    for col in ["G数", "差枚", "rate_denominator", "台番"]:
        if col not in df.columns:
            raise KeyError(f"列 {col} が見つかりません。")

    cond_g    = (df["G数"] >= G_MIN).fillna(False)
    cond_rate = (df["rate_denominator"] <= RATE_MAX_DEN).fillna(False)
    cond_s    = (df["差枚"] < SAMAI_UPPER).fillna(False)

    # 新しく追加した台番の条件
    cond_dai_ban = df["台番"].isin(SPECIFIC_DAI_BAN_LIST).fillna(False)
    
    # 全ての条件をANDで結合
    out = df.loc[cond_g & cond_rate & cond_s & cond_dai_ban].copy()
    
    # 見やすく：良い順（分母が小さいほど良い）→ G数大きい順
    out = out.sort_values(["rate_denominator", "G数"], ascending=[True, False])
    return out

def attach_next_day_rows(today_date: datetime) -> pd.DataFrame:
    """当日の抽出結果に、翌日の同台番のカラムを _next で横付け"""
    df_today = read_day(today_date)
    filtered = filter_by_conditions(df_today)

    next_date = today_date + timedelta(days=1)

    # 翌日のCSV（無ければ台番だけの枠を作り、結合後はNaNのまま残す）
    try:
        df_next = read_day(next_date)
    except FileNotFoundError:
        df_next = pd.DataFrame({"台番": filtered["台番"].unique()})

    # 翌日から拾う列
    keep_cols = ["台番", "差枚", "G数", "BB", "RB", "出率", "BB率", "RB率", "合成"]
    exist_cols = [c for c in keep_cols if c in df_next.columns]
    df_next_small = df_next[exist_cols].copy()
    df_next_small = df_next_small.rename(columns={c: f"{c}_next" for c in exist_cols if c != "台番"})

    merged = filtered.merge(df_next_small, on="台番", how="left")
    merged.insert(0, "date", today_date.strftime("%Y-%m-%d"))
    merged.insert(1, "next_date", next_date.strftime("%Y-%m-%d"))
    return merged

def daterange(start_date: datetime, end_date: datetime):
    d = start_date
    while d <= end_date:
        yield d
        d += timedelta(days=1)

# ===== メイン（まとめのみ保存） =====
def run_range(start_date_str: str, end_date_str: str) -> pd.DataFrame:
    start = datetime.strptime(start_date_str, "%Y-%m-%d")
    end   = datetime.strptime(end_date_str,   "%Y-%m-%d")

    all_rows = []
    for d in daterange(start, end):
        path = compose_path(d)
        if not os.path.exists(path):
            msg = f"[SKIP] {os.path.basename(path)} が見つかりません。"
            if SKIP_MISSING_DAYS:
                print(msg)
                continue
            else:
                raise FileNotFoundError(msg)
        try:
            day_result = attach_next_day_rows(d)
            all_rows.append(day_result)
        except Exception as e:
            print(f"[ERROR] {d.date()} の処理でエラー: {e}")

    if not all_rows:
        print("[INFO] 条件に一致する行がありませんでした。")
        return pd.DataFrame()

    total = pd.concat(all_rows, ignore_index=True)

    # *_next を整数（nullable Int64）に揃える（欠損は <NA> のまま）
    for c in ["差枚_next", "G数_next", "BB_next", "RB_next"]:
        if c in total.columns:
            total[c] = pd.to_numeric(total[c], errors="coerce").astype("Int64")

    # 出力先フォルダ（notebook\result）に保存
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    out_total = os.path.join(
        OUTPUT_DIR,
        f"spec-{start:%Y%m%d}-{end:%Y%m%d}.csv"
    )
    total.to_csv(out_total, index=False, encoding="utf-8-sig")
    print(f"[SAVE] {out_total} (rows={len(total)})")
    return total

if __name__ == "__main__":
    _ = run_range(START_DATE, END_DATE)

[SAVE] C:\Users\akafu\analytics\aim_machi\notebook\result\spec-20250613-20250915.csv (rows=72)


In [7]:
import os
import glob
import pandas as pd
from pathlib import Path

# 「平均」ラベルの行を集めて notebook/result/ave.csv に保存

def find_dir_up(start: Path, target_name: str) -> Path | None:
    """start から親方向へ target_name のディレクトリを探す"""
    cur = start.resolve()
    while True:
        cand = cur / target_name
        if cand.is_dir():
            return cand
        if cur.parent == cur:
            return None
        cur = cur.parent

def main():
    # 実行ディレクトリ（.py でも .ipynb でも対応）
    base_dir = Path(__file__).parent if "__file__" in globals() else Path.cwd()

    # 上方向にたどって data を特定（見つからなければ base_dir/data、無ければ base_dir）
    data_dir = find_dir_up(base_dir, "data") or (base_dir / "data")
    if not data_dir.is_dir():
        data_dir = base_dir  # 最後の保険

    # プロジェクトルートを推定（data のひとつ上）
    project_root = data_dir.parent if data_dir.name.lower() == "data" else base_dir

    # 出力先は notebook/result（なければ作る）。無ければ base_dir に落とす。
    output_dir = project_root / "notebook" / "result"
    try:
        output_dir.mkdir(parents=True, exist_ok=True)
    except Exception:
        output_dir = base_dir

    # CSV ファイル一覧（集計系は除外）
    files = sorted(
        f for f in glob.glob(str(data_dir / "*.csv"))
        if not Path(f).name.startswith(("ave", "mean_rows", "last_row_averages", "last_rows",
                                        "Filtered With Next Range", "aggregated"))
    )

    extracted = []
    for fp in files:
        # 文字コードを優先順で試す
        try:
            df = pd.read_csv(fp, encoding="utf-8-sig")
        except Exception:
            df = pd.read_csv(fp, encoding="cp932", errors="replace")
            
        if df.empty:
            continue

        # === 💡プロの修正：より堅牢な「平均」抽出ロジック ===
        # 「台番」列があればそこを優先して探し、無ければ全列を探す
        target_cols = ['台番'] if '台番' in df.columns else df.columns
        
        mean_rows = pd.DataFrame()
        for col in target_cols:
            # NaNを回避し、空白を除去してから「平均」の文字が含まれるか判定
            mask = df[col].astype(str).str.strip().str.contains("平均", na=False)
            if mask.any():
                mean_rows = df[mask]
                break  # 見つかったら他の列は探さずにループ終了
        
        # それでも見つからなければ次のファイルへ
        if mean_rows.empty:
            continue
        # =================================================

        # 元ファイル名を先頭列に付与
        mean_rows = mean_rows.copy()
        mean_rows.insert(0, "filename", Path(fp).name)
        extracted.append(mean_rows)

    if not extracted:
        print("「平均」ラベルの行が見つかりませんでした。")
        return

    result = pd.concat(extracted, ignore_index=True)
    print(f" {len(extracted)} 件のファイルから「平均」行を抽出しました！")
    print(result.head()) # 確認のために少しだけ表示

    out_fp = output_dir / "ave.csv"
    result.to_csv(out_fp, index=False, encoding="utf-8-sig")
    print(f"\nSaved mean rows to {out_fp}")

if __name__ == "__main__":
    main()

 77 件のファイルから「平均」行を抽出しました！
                 filename  台番   差枚    G数      出率  BB  RB     合成    BB率    RB率
0  aim-2025-07-01-Tue.csv  平均  164  2982  101.8%  11  10  1/142  1/261  1/311
1  aim-2025-07-02-Wed.csv  平均 -166  3088   98.2%  11   9  1/155  1/285  1/339
2  aim-2025-07-03-Thu.csv  平均   -2  3311  100.0%  12  10  1/152  1/269  1/347
3  aim-2025-07-04-Fri.csv  平均 -159  3644   98.5%  13  10  1/155  1/276  1/356
4  aim-2025-07-05-Sat.csv  平均   84  5879  100.5%  22  18  1/147  1/267  1/328

Saved mean rows to C:\Users\akafu\analytics\aim_machi\notebook\result\ave.csv
